# Assignment 1 – Backprop

In [1]:
#@title Library Imports [do not change]

import importlib 
!git clone https://www.github.com/rycolab/intro-nlp-f22.git
utils = importlib.import_module("intro-nlp-f22.assignment_1.utils")

import re
import random
from collections import defaultdict
import itertools
from abc import ABC, abstractmethod
import math

Cloning into 'intro-nlp-f22'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 32 (delta 6), reused 30 (delta 4), pack-reused 0
Unpacking objects: 100% (32/32), done.


In [2]:
#@title Select and Parse Math Problems [do not change]

#@markdown select math problem

math_problem_i = "3" #@param [0,1,2,3]
math_problem = utils.MATH_PROBLEMS[int(math_problem_i)]
print(math_problem)

parser = utils.Parser()
infix, in_vars = parser.parse(math_problem["problem"], in_vars = math_problem["in_vars"])
print(infix, in_vars)

{'problem': 'z + sin(x^(2) + (y * exp(z)))', 'in_vars': {'x': 2.0, 'y': -1.0, 'z': 0.0}, 'output': 0.14, 'derivative': {'x': -3.96, 'y': -0.99, 'z': 1.99}}
['z', '+', ['sin', [['x', '^', 2], '+', ['y', '*', ['exp', 'z']]]]] {'x': 2.0, 'y': -1.0, 'z': 0.0}


In [3]:
#@title ToDo1: Building

class Node:
    def __init__(
        self,
        label=None,
        value=None,
        operator=None,
        parent=None,
        children=None,
        derivative=None,
        is_var=False,
        is_num=False,
    ):
        self.operator = operator
        self.value = value
        self.derivative = derivative
        self.children = children
        self.parent = parent
        self.label = label
        self.is_var = is_var
        self.is_num = is_num

    def __str__(self):  # To check correctness
        if self.is_var:
            return self.label
        elif self.value:
            return str(self.value)
        else:  # List
            if len(self.children) == 1:
                return f"({self.operator}({self.children[0]}))"
            elif len(self.children) == 2:
                return f"({self.children[0]} {self.operator} {self.children[1]})"
            else:
                raise NotImplementedError


def infix_to_node(infix) -> Node:
    """Convert an 'infix' to a nested Node object.

    Recursively instantiate Node objects according to the
    nested structure of infix.
    E.g if we have [['exp', 'x'], '-', ['y', '+', '2']]
    Then we get Node(label='-', ..., children=[Node(label='exp', ...), Node(label='+', ...)])

    Parameters
    ----------
    infix : list
        Nested list of operations, variables and numbers.
        Leaf node lists must be max length 3.

    Returns
    -------
    Node
        Nested Node object representing the graph.

    Raises
    ------
    NotImplementedError:
        If infix contains an item that is not an int, float, str or list.
    """
    if isinstance(infix, int) or isinstance(infix, float):
        return Node(value=infix, is_num=True, label=str(infix))
    elif isinstance(infix, str):
        return Node(label=infix, is_var=True)
    elif isinstance(infix, list):
        if len(infix) == 2:
            node = Node(
                operator=infix[0],
                label=infix[0],
                children=[
                    infix_to_node(infix=infix[1]),
                ],
            )
            node.children[0].parent = node
            return node
        elif len(infix) == 3:
            node = Node(
                operator=infix[1],
                label=infix[1],
                children=[
                    infix_to_node(infix=infix[0]),
                    infix_to_node(infix=infix[2]),
                ],
            )
            node.children[0].parent = node
            node.children[1].parent = node
            return node
        else:
            raise NotImplementedError
    else:
        raise NotImplementedError




class Builder():

    def __init__(self, infix: list, in_vars: dict = {}):
        """
        infix: list of infix notation parse, e.g. [['exp', 2], '-', 3]
        in_vars: dict of input variables to ensure they are not used as intermediate or output variables
        RETURN: computation graph in a data structure of your choosing
        """

        ## some alphabetical vars to use as intermediate and output variables minus the input vars to avoid duplicates
        avail_vars = list(map(chr, range(97, 123))) + list(map(chr, range(945, 969)))
        if len(in_vars.keys()) > 0:
            avail_vars = set(avail_vars) - set(in_vars)
        self.avail_vars = sorted(list(set(avail_vars)), reverse=True)

        self.infix = infix
        self.graph = infix_to_node(infix)


if __name__ == '__main__':

    g = Builder(infix, in_vars)
    print(g.avail_vars)
    print(g.infix)
    print(g.graph)

['ψ', 'χ', 'φ', 'υ', 'τ', 'σ', 'ς', 'ρ', 'π', 'ο', 'ξ', 'ν', 'μ', 'λ', 'κ', 'ι', 'θ', 'η', 'ζ', 'ε', 'δ', 'γ', 'β', 'α', 'w', 'v', 'u', 't', 's', 'r', 'q', 'p', 'o', 'n', 'm', 'l', 'k', 'j', 'i', 'h', 'g', 'f', 'e', 'd', 'c', 'b', 'a']
['z', '+', ['sin', [['x', '^', 2], '+', ['y', '*', ['exp', 'z']]]]]
(z + (sin(((x ^ 2) + (y * (exp(z)))))))


In [4]:

class Operator(ABC):
    @abstractmethod
    def f(self, a, b=None) -> float:
        raise NotImplementedError()
        return f_res

    @abstractmethod
    def df(self, a, b=None) -> list:
        raise NotImplementedError()
        return [df_res]


class Exp(Operator):
    def f(self, a, b=None):
        return math.exp(a)

    def df(self, a, b=None):
        return [math.exp(a)]


class Log(Operator):
    # natural logarithm

    def f(self, a, b=None):
        return math.log(a)

    def df(self, a, b=None):
        return [1 / a]


class Mult(Operator):
    def f(self, a, b):
        return a * b

    def df(self, a, b=None):
        return [b, a]


class Div(Operator):
    def f(self, a, b):
        return a / b

    def df(self, a, b):
        return [1 / b, (-a) / (b**2)]


class Add(Operator):
    def f(self, a, b):
        if not b:
            return a  # f(x) = +x unary case
        if not a:
            return b
        else:
          return a + b

    def df(self, a, b=None):
        if not b:
            return [1]
        else:
            return [1, 1]


class Sub(Operator):
    def f(self, a, b=None):
        if not b:
            return -a
        else:
            return a - b

    def df(self, a, b=None):
        if not b:
            return [-1]  # f(x) = -x unary case
        else:
            return [1, -1]


class Pow(Operator):
    def f(self, a, b):
        return a**b

    def df(self, a, b):
        if a <= 0:  # work-around: treat as unary operation if -a^b
            return [b * (a ** (b - 1))]
        else:
            return [b * (a ** (b - 1)), (a**b) * math.log(a)]


class Sin(Operator):
    def f(self, a, b=None):
        return math.sin(a)

    def df(self, a, b=None):
        return [math.cos(a)]


class Cos(Operator):
    def f(self, a, b=None):
        return math.cos(a)

    def df(self, a, b=None):
        return [-1 * math.sin(a)]




In [5]:
#@title ToDo 3: Executing

class Executor:
    def __init__(self, graph: Node, in_vars: dict = {}):
        """
        graph: computation graph in a data structure of your choosing
        in_vars: dict of input variables, e.g. {"x": 2.0, "y": -1.0}
        """
        self.graph = graph
        self.in_vars = in_vars
        self.fn_map = {
            "log": Log(),
            "exp": Exp(),
            "+": Add(),
            "-": Sub(),
            "^": Pow(),
            "sin": Sin(),
            "cos": Cos(),
            "*": Mult(),
            "/": Div(),
        }
        self.output = -1
        self.derivative = {}

    def forward(self) -> None:
        """Populate the values of all nodes in self.graph recursively."""
        def eval_node(node):
            if node.label in self.in_vars.keys():
                node.value = self.in_vars[node.label]
                return node.value

            elif node.is_num:
                return node.value

            else:
                node.value = self.fn_map[node.operator].f(
                    *[eval_node(child) for child in node.children]
                )
                return node.value

        self.output = eval_node(node=self.graph)

    def backward(self) -> None:
        """Populate self.derivative dictionary using backpropgation."""
        self.derivative = {key: 0.0 for key in self.in_vars.keys()}

        def previsitor(node: Node) -> None:
            if node.is_var:
                self.derivative[node.label] += node.derivative
            elif node.is_num:
                pass
            else:
                if not node.parent:
                    node.derivative = 1

                jacobian = [
                    node.derivative * df
                    for df in self.fn_map[node.operator].df(
                        *[child.value for child in node.children]
                    )
                ]
                for idx, child in enumerate(node.children):
                    child.derivative = jacobian[idx]
                    previsitor(child)

        previsitor(self.graph)




if __name__ == '__main__':
  e = Executor(g.graph, in_vars=in_vars)
  e.forward()
  e.backward()
  print(e.output)
  print(e.derivative)

0.1411200080598672
{'x': -3.9599699864017817, 'y': -0.9899924966004454, 'z': 1.9899924966004454}


In [6]:
#@title Test Function for Debugging [do not change]

utils.test_backprop(Builder, Executor, math_problem)


0: problem: z + sin(x^(2) + (y * exp(z))), in_vars: {'x': 2.0, 'y': -1.0, 'z': 0.0}
SUCCESS output: 0.14
SUCCESS derivative: {'x': -3.96, 'y': -0.99, 'z': 1.99}


In [7]:
#@title Test Function for Grading [do not change]

utils.test_backprop(Builder, Executor)


0: problem: x/y, in_vars: {'x': 1.0, 'y': 1.0}
SUCCESS output: 1.0
SUCCESS derivative: {'x': 1.0, 'y': -1.0}

1: problem: exp(x) - (y * 2), in_vars: {'x': 2.0, 'y': -2.0}
SUCCESS output: 11.39
SUCCESS derivative: {'x': 7.39, 'y': -2.0}

2: problem: (x^2 - 1) * (y+2), in_vars: {'x': 3.0, 'y': 2.0}
SUCCESS output: 32.0
SUCCESS derivative: {'x': 24.0, 'y': 8.0}

3: problem: z + sin(x^(2) + (y * exp(z))), in_vars: {'x': 2.0, 'y': -1.0, 'z': 0.0}
SUCCESS output: 0.14
SUCCESS derivative: {'x': -3.96, 'y': -0.99, 'z': 1.99}
